In [1]:
import pandas as pd
from tensorflow.keras.models import load_model
import pickle
import tensorflow as tf
import numpy as np

In [2]:
### load the trained model, 
model = load_model('model.h5')

#load encoders and decoders
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo = pickle.load(file)

#load scaler
with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [3]:
# Example input data
input_data = {
    'CreditScore': 500,
    'Geography': 'France',
    'Gender': 'Female',
    'Age': 30,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [4]:
## onehot encoder geography

geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

d:\Pavan\DeepLearning\tensorflow_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [5]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,500,France,Female,30,3,60000,2,1,1,50000


In [6]:
##label encoder for gender

input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,500,France,0,30,3,60000,2,1,1,50000


In [7]:
## concat onehot enocder

input_df = pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,500,0,30,3,60000,2,1,1,50000,1.0,0.0,0.0


In [8]:
##scaling the inputdata
input_scaled = scaler.transform(input_df)
input_scaled

array([[-1.5563406 , -1.09003844, -0.84918282, -0.69146874, -0.26845961,
         0.79740451,  0.64376017,  0.97067965, -0.87572587,  0.99401789,
        -0.57773517, -0.57234647]])

In [9]:
##predict churn
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 88ms/step


array([[0.02474219]], dtype=float32)

In [10]:
prediction_prob = prediction[0][0]

In [11]:
prediction_prob

0.024742186

In [12]:
if prediction_prob >0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")

The customer is not likely to churn.
